# Importations

In [ ]:
!git clone https://github.com/z-mahmud22/Mask-RCNN_TF2.14.0.git

In [ ]:
import sys
print(sys.version)


In [ ]:
# installation du requirement.text
!pip install cython==3.0.5 h5py==3.9.0 imgaug==0.4.0 ipython==7.34.0 ipython-genutils==0.2.0 ipython-sql==0.5.0 keras==2.14.0 matplotlib==3.7.1 numpy==1.23.5 opencv-contrib-python==4.8.0.76 opencv-python==4.8.0.76 pillow==9.4.0 scikit-image==0.19.3 scipy==1.11.3 tensorboard==2.14.1 tensorflow[and-cuda]==2.14.0


In [ ]:
# Se déplacer vers le répertoire contenant setup.py
%cd /content/Mask-RCNN_TF2.14.0

# Installer le package et ses dépendances
!pip install .


In [ ]:
import os
from pycocotools.coco import COCO

# ici mon chemin d'accès est celui de colab où j'importe mes images et annotations
data_dir = '/content'



importation des poids de coco

In [ ]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5


In [ ]:
#première config
from mrcnn.config import Config

class CustomConfig(Config):
    NAME = "baby"
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 1 + 2  # Background + "nose" + "face"
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 50
    LEARNING_RATE = 0.001

    # Dimensions des images
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_SHAPE = [512, 512, 3]

    # Utilisation d'un mini masque pour économiser de la mémoire
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)

    # Paramètres du RPN (Region Proposal Network)
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 200
    GPU_COUNT = 1
# Instantiate your custom configuration
config = CustomConfig()

Passage des annotations au bon format (les deux formats sont disponibles dans mon repo Git donc vous pouvez sauter cette étape)

In [ ]:
import json

# Charger les données d'entrée depuis votre chemin
input_path = "/content/images/train/train_annotations_json.json"
output_path = "/content/images/train/train_annotations_coco.json"

# Charger les données d'entrée
with open(input_path, "r") as file:
    input_data = json.load(file)

# Initialiser les structures pour COCO
coco_format = {
    "info": {
        "year": 2024,
        "version": "1.0",
        "description": "Dataset COCO pour Baby Dataset",
        "contributor": "",
        "url": "",
        "date_created": "2024-11-17"
    },
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "nose", "supercategory": "baby"},
        {"id": 2, "name": "face", "supercategory": "baby"}
    ]
}

# Variables auxiliaires
image_id = 1
annotation_id = 1
category_map = {"nose": 1, "face": 2}

# Parcourir les images
for image_key, image_data in input_data.items():
    # Ajouter les métadonnées de l'image
    coco_format["images"].append({
        "id": image_id,
        "width": 640,  # Remplacer par la largeur réelle si disponible
        "height": 640,  # Remplacer par la hauteur réelle si disponible
        "file_name": image_data["filename"],
        "license": 0,
        "date_captured": ""
    })

    # Ajouter les annotations
    for region in image_data["regions"]:
        shape = region["shape_attributes"]
        attributes = region["region_attributes"]
        category_name = list(attributes.values())[0]  # Exemple : "nose" ou "face"

        if category_name in category_map:
            category_id = category_map[category_name]
            x, y, width, height = shape["x"], shape["y"], shape["width"], shape["height"]

            # Calcul du segment
            segmentation = [
                [x, y, x + width, y, x + width, y + height, x, y + height]
            ]

            # Ajouter l'annotation
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x, y, width, height],
                "area": width * height,
                "iscrowd": 0,
                "segmentation": segmentation
            })
            annotation_id += 1

    image_id += 1

# Sauvegarder dans un fichier JSON au format COCO
with open(output_path, "w") as output_file:
    json.dump(coco_format, output_file, indent=4)

print(f"Fichier converti enregistré à : {output_path}")


In [ ]:
import json

# Charger les données d'entrée depuis votre chemin
input_path = "/content/images/test/test_annotations_json.json"

output_path = "/content/images/test/test_annotations_coco.json"

# Charger les données d'entrée
with open(input_path, "r") as file:
    input_data = json.load(file)

# Initialiser les structures pour COCO
coco_format = {
    "info": {
        "year": 2024,
        "version": "1.0",
        "description": "Dataset COCO pour Baby Dataset",
        "contributor": "",
        "url": "",
        "date_created": "2024-11-17"
    },
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "nose", "supercategory": "baby"},
        {"id": 2, "name": "face", "supercategory": "baby"}
    ]
}

# Variables auxiliaires
image_id = 1
annotation_id = 1
category_map = {"nose": 1, "face": 2}

# Parcourir les images
for image_key, image_data in input_data.items():
    # Ajouter les métadonnées de l'image
    coco_format["images"].append({
        "id": image_id,
        "width": 640,  # Remplacer par la largeur réelle si disponible
        "height": 640,  # Remplacer par la hauteur réelle si disponible
        "file_name": image_data["filename"],
        "license": 0,
        "date_captured": ""
    })

    # Ajouter les annotations
    for region in image_data["regions"]:
        shape = region["shape_attributes"]
        attributes = region["region_attributes"]
        category_name = list(attributes.values())[0]  # Exemple : "nose" ou "face"

        if category_name in category_map:
            category_id = category_map[category_name]
            x, y, width, height = shape["x"], shape["y"], shape["width"], shape["height"]

            # Calcul du segment
            segmentation = [
                [x, y, x + width, y, x + width, y + height, x, y + height]
            ]

            # Ajouter l'annotation
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x, y, width, height],
                "area": width * height,
                "iscrowd": 0,
                "segmentation": segmentation
            })
            annotation_id += 1

    image_id += 1

# Sauvegarder dans un fichier JSON au format COCO
with open(output_path, "w") as output_file:
    json.dump(coco_format, output_file, indent=4)

print(f"Fichier converti enregistré à : {output_path}")


In [ ]:
import json

# Charger les données d'entrée depuis votre chemin
input_path = "/content/images/valid/valid_annotations_json.json"
output_path = "/content/images/valid/valid_annotations_coco.json"

# Charger les données d'entrée
with open(input_path, "r") as file:
    input_data = json.load(file)

# Initialiser les structures pour COCO
coco_format = {
    "info": {
        "year": 2024,
        "version": "1.0",
        "description": "Dataset COCO pour Baby Dataset",
        "contributor": "",
        "url": "",
        "date_created": "2024-11-17"
    },
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "nose", "supercategory": "baby"},
        {"id": 2, "name": "face", "supercategory": "baby"}
    ]
}

# Variables auxiliaires
image_id = 1
annotation_id = 1
category_map = {"nose": 1, "face": 2}

# Parcourir les images
for image_key, image_data in input_data.items():
    # Ajouter les métadonnées de l'image
    coco_format["images"].append({
        "id": image_id,
        "width": 640,  # Remplacer par la largeur réelle si disponible
        "height": 640,  # Remplacer par la hauteur réelle si disponible
        "file_name": image_data["filename"],
        "license": 0,
        "date_captured": ""
    })

    # Ajouter les annotations
    for region in image_data["regions"]:
        shape = region["shape_attributes"]
        attributes = region["region_attributes"]
        category_name = list(attributes.values())[0]  # Exemple : "nose" ou "face"

        if category_name in category_map:
            category_id = category_map[category_name]
            x, y, width, height = shape["x"], shape["y"], shape["width"], shape["height"]

            # Calcul du segment
            segmentation = [
                [x, y, x + width, y, x + width, y + height, x, y + height]
            ]

            # Ajouter l'annotation
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x, y, width, height],
                "area": width * height,
                "iscrowd": 0,
                "segmentation": segmentation
            })
            annotation_id += 1

    image_id += 1

# Sauvegarder dans un fichier JSON au format COCO
with open(output_path, "w") as output_file:
    json.dump(coco_format, output_file, indent=4)

print(f"Fichier converti enregistré à : {output_path}")


Définition des fonctions principales

In [ ]:
from mrcnn.utils import Dataset
from mrcnn.model import *
from pycocotools.coco import COCO
import numpy as np
import os
import cv2
from pycocotools import mask as maskUtils
config = CustomConfig()

class BabyDataset(Dataset):
    def load_baby(self, dataset_dir, subset):
        """Charge le dataset de bébés à partir du répertoire spécifié."""
        assert subset in ["train", "valid","test"]
        dataset_path = os.path.join(dataset_dir, subset)

        # Charger le fichier d'annotations approprié
        annotations_file = os.path.join(dataset_path, f"{subset}_annotations_coco.json")
        if not os.path.exists(annotations_file):
            raise FileNotFoundError(f"Le fichier d'annotations {annotations_file} est introuvable.")

        # Charger les annotations avec COCO directement à partir du chemin du fichier
        coco = COCO(annotations_file)

        # Ajouter des classes (basé sur le fichier d'annotations)
        for cat in coco.loadCats(coco.getCatIds()):
            self.add_class("baby", cat['id'], cat['name'])

        # Ajouter les images
        image_ids = list(coco.imgs.keys())
        for image_id in image_ids:
            image_info = coco.loadImgs(image_id)[0]
            image_path = os.path.join(dataset_path, image_info['file_name'])

            if not os.path.exists(image_path):
                print(f"Image {image_path} manquante.")
                continue

            # Ajouter l'image au dataset
            self.add_image(
                "baby",
                image_id=image_id,
                path=image_path,
                width=image_info['width'],
                height=image_info['height'],
                annotations_path=annotations_file
            )



    def load_mask(self, image_id):
      """Génère les masques pour une image donnée."""
      image_info = self.image_info[image_id]
      coco = COCO(image_info['annotations_path'])
      annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_info['id'], iscrowd=False))

      height, width = image_info['height'], image_info['width']

      # Initialiser le masque avec le bon type de données
      mask = np.zeros((config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], len(annotations)), dtype=np.uint8)  # Changed to config.IMAGE_SHAPE
      class_ids = []

      for i, ann in enumerate(annotations):
          rle = coco.annToRLE(ann)
          m = maskUtils.decode(rle)

          # Vérifier le type du masque après le décodage
          assert m.dtype == np.uint8 or m.dtype == np.bool_, f"Mask dtype after decode is {m.dtype}"

          # Redimensionner le masque pour qu'il corresponde à la taille de l'image
          if m.shape[0] != height or m.shape[1] != width:
              m = cv2.resize(m, (width, height), interpolation=cv2.INTER_NEAREST)

          # Redimensionner le masque pour qu'il corresponde aux dimensions attendues par le modèle
          m = cv2.resize(m, (config.IMAGE_SHAPE[1], config.IMAGE_SHAPE[0]), interpolation=cv2.INTER_NEAREST)

          # Vérifier le type du masque après le redimensionnement
          assert m.dtype == np.uint8, f"Mask dtype after resize is {m.dtype}"

          # Convertir en uint8 si nécessaire
          mask[:, :, i] = m.astype(np.uint8)

          # Vérifier le type du masque après la conversion
          assert mask[:, :, i].dtype == np.uint8, f"Mask dtype after assignment is {mask[:, :, i].dtype}"

          # Ajouter la classe correspondante
          class_id = self.map_source_class_id(f"baby.{ann['category_id']}")
          class_ids.append(class_id)

      # Move the conversion to NumPy array outside the loop
      class_ids = np.array(class_ids, dtype=np.int32)
      return mask, class_ids



    def image_reference(self, image_id):
        """Retourne le chemin de l'image."""
        return self.image_info[image_id]["path"]

# Charger le dataset d'entraînement
dataset_dir = "/content/images"
dataset_train = BabyDataset()
dataset_train.load_baby(dataset_dir, 'train')
dataset_train.prepare()
print(f"Nombre d'images dans le dataset d'entraînement: {len(dataset_train.image_ids)}")

# Charger le dataset de validation
dataset_val = BabyDataset()
dataset_val.load_baby(dataset_dir, 'valid')
dataset_val.prepare()
print(f"Nombre d'images dans le dataset de validation: {len(dataset_val.image_ids)}")


In [ ]:
# Tester le chargement du masque pour une image
image_id = dataset_train.image_ids[0]
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)

print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")
print(f"Classes: {class_ids}")


verification importation image et masque

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import random

def display_image_with_masks(image, mask, class_ids, class_names):
    """
    Affiche l'image avec les masques superposés et les classes annotées.
    """
    plt.figure(figsize=(8, 8))
    plt.imshow(image)

    # Générer des couleurs aléatoires pour chaque classe
    colors = [(random.random(), random.random(), random.random()) for _ in range(len(class_ids))]

    # Superposer chaque masque sur l'image
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        color = colors[i]

        # Superposer le masque en semi-transparent
        plt.imshow(m, cmap='cool', alpha=0.4)

        # Ajouter une légende avec le nom de la classe
        class_id = class_ids[i]
        class_name = class_names[class_id]
        plt.text(10, 30 + i * 20, f"{class_name} (ID {class_id})", color=color, fontsize=12, backgroundcolor='black')

    plt.axis('off')
    plt.show()

# Charger une image et ses masques
image_id = dataset_train.image_ids[0]
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)

# Afficher l'image avec les masques superposés
display_image_with_masks(image, mask, class_ids, ["BG", "nose", "face"])




In [ ]:
dataset_dir = "/content/images"

# Charger le dataset d'entraînement
dataset_train = BabyDataset()
dataset_train.load_baby(dataset_dir, 'train')
dataset_train.prepare()
print(f"Nombre d'images dans le dataset d'entraînement: {len(dataset_train.image_ids)}")
# Charger le dataset de validation
dataset_val = BabyDataset()
dataset_val.load_baby(dataset_dir, 'valid')
dataset_val.prepare()
print(f"Nombre d'images dans le dataset de validation: {len(dataset_val.image_ids)}")


A partir d'ici tout est en double car je teste deux configuration des hyperparamètres afin de voir lesquels sont meilleurs pour mon model avec mon dataset

In [ ]:
class BabyConfig(Config):
    NAME = "baby"
    NUM_CLASSES = 1 + 2  # Fond + nose et face
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 10
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 200
    MAX_GT_INSTANCES = 50
    DETECTION_MAX_INSTANCES = 50
    DETECTION_MIN_CONFIDENCE = 0.9
    DETECTION_NMS_THRESHOLD = 0.3


In [ ]:
class BabyConfig_1(Config):
    NAME = "baby"
    NUM_CLASSES = 1 + 2  # Fond + nose et face
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 10
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 200
    MAX_GT_INSTANCES = 50
    DETECTION_MAX_INSTANCES = 50
    DETECTION_MIN_CONFIDENCE = 0.9
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.001  # Taux d'apprentissage initial
    # Taille des lots
    BATCH_SIZE = GPU_COUNT * IMAGES_PER_GPU



création du model et entrainement avec la première configuration

In [ ]:
from mrcnn.model import MaskRCNN
config = BabyConfig()
# Créer le modèle en mode entraînement
model = MaskRCNN(mode="training", config=config, model_dir='/content/logs')

# Charger un modèle pré-entraîné (par exemple COCO)
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# Vérifier les types de données avant l'entraînement
for image_id in dataset_train.image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    assert image.dtype == np.uint8, f"Training image dtype is {image.dtype}"
    assert mask.dtype == np.uint8, f"Training mask dtype is {mask.dtype}"

# Entraîner le modèle
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='heads')


In [ ]:
history_path = os.path.join('/content', 'training_logs_model.txt')
with open(history_path, 'w') as log_file:
    log_file.write(str(model.keras_model.history.history))  


import matplotlib.pyplot as plt

loss = model.keras_model.history.history['loss']
val_loss = model.keras_model.history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Passer en mode inférence
class InferenceConfig(Config):
    NAME = "Baby"
    NUM_CLASSES = 1 + 2
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    BATCH_SIZE = 1

inference_config = InferenceConfig()


model = MaskRCNN(mode="inference", config=inference_config, model_dir='/content/logs')

import os
if os.path.exists('/content/logs/mask_rcnn_baby.h5'):
  model.load_weights('/content/logs/mask_rcnn_baby.h5', by_name=True)
else:
  print("Weight file not found. Please ensure the model has been trained and the weights are saved to the correct location.")
  

# Charger une image pour effectuer des prédictions
image_id = dataset_val.image_ids[0]
image = dataset_val.load_image(image_id)
mask, class_ids = dataset_val.load_mask(image_id)

# Effectuer la prédiction
results = model.detect([image], verbose=1)
r = results[0]

# Afficher les résultats
import mrcnn.visualize
mrcnn.visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                  dataset_val.class_names, r['scores'])

In [ ]:
dataset_dir = "/content/images/"

# Charger le dataset d'entraînement
dataset_test = BabyDataset()
dataset_test.load_baby( dataset_dir, 'test')
dataset_test.prepare()
print(f"Nombre d'images dans le dataset de test: {len(dataset_test.image_ids)}")

In [ ]:

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


class InferenceConfig(Config):
    NAME = "Baby"
    NUM_CLASSES = 1 + 2
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_model = MaskRCNN(mode="inference", config=inference_config, model_dir='/content')


inference_model.load_weights(model.find_last(), by_name=True)


true_labels = []
pred_labels = []

for image_id in dataset_test.image_ids:
    
    true_masks, true_classes = dataset_test.load_mask(image_id)

    
    image = dataset_test.load_image(image_id)
    pred = inference_model.detect([image], verbose=0)[0]

    
    if len(true_classes) == len(pred['class_ids']):
        true_labels.extend(true_classes)
        pred_labels.extend(pred['class_ids'])
    else:
        
        true_labels.extend(true_classes)
        pred_labels.extend(pred['class_ids'])

       
        if len(true_classes) > len(pred['class_ids']):
            pred_labels.extend([0] * (len(true_classes) - len(pred['class_ids'])))
        elif len(pred['class_ids']) > len(true_classes):
            true_labels.extend([0] * (len(pred['class_ids']) - len(true_classes)))


cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1])  
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['background', 'baby'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


from mrcnn.visualize import display_instances
from mrcnn import visualize

import random


for i in random.sample(list(dataset_test.image_ids), 5):
    image = dataset_test.load_image(i)  
    results = inference_model.detect([image], verbose=0)  

    
    visualize.display_instances(
        image,
        results[0]['rois'],
        results[0]['masks'],
        results[0]['class_ids'],
        dataset_test.class_names,
        results[0]['scores']
    )


In [ ]:
from mrcnn.utils import compute_ap
import numpy as np


APs = []


for image_id in dataset_test.image_ids:
    
    image = dataset_test.load_image(image_id)

    
    gt_mask, gt_class_ids = dataset_test.load_mask(image_id)

    
    gt_bbox = utils.extract_bboxes(gt_mask)

    
    results = inference_model.detect([image], verbose=0)
    r = results[0]

    
    if r['masks'].size == 0:
        continue

    
    pred_masks = r['masks']

    
    if pred_masks.shape[0] != gt_mask.shape[0] or pred_masks.shape[1] != gt_mask.shape[1]:
        pred_masks_resized = []
        for i in range(pred_masks.shape[-1]):
            mask = pred_masks[:, :, i]
            mask_resized = cv2.resize(mask.astype(np.uint8),
                                    (gt_mask.shape[1], gt_mask.shape[0]),
                                    interpolation=cv2.INTER_NEAREST)
            pred_masks_resized.append(mask_resized)
        pred_masks = np.stack(pred_masks_resized, axis=-1)

    
    if gt_class_ids.size == 0 or r['class_ids'].size == 0:
        continue

    
    AP, precisions, recalls, overlaps = compute_ap(
        gt_bbox,          # Ground truth bounding boxes
        gt_class_ids,     # Ground truth class IDs
        gt_mask,          # Ground truth masks
        r['rois'],        # Predicted bounding boxes
        r['class_ids'],   # Predicted class IDs
        r['scores'],      # Predicted scores (was missing in original)
        pred_masks        # Predicted masks
    )

    APs.append(AP)


if len(APs) > 0:
    mAP = np.mean(APs)
    print(f"Mean Average Precision (mAP): {mAP:.4f}")
else:
    print("No valid predictions found for evaluation")


In [ ]:
from mrcnn.utils import compute_ap
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import cv2


APs = []
all_true_labels = []
all_pred_labels = []


for image_id in dataset_test.image_ids:
    
    image = dataset_test.load_image(image_id)
    
    gt_mask, gt_class_ids = dataset_test.load_mask(image_id)

    gt_bbox = utils.extract_bboxes(gt_mask)

    
    results = inference_model.detect([image], verbose=0)
    r = results[0]

    
    if r['masks'].size == 0:
        continue

    
    pred_masks = r['masks']

    
    if pred_masks.shape[0] != gt_mask.shape[0] or pred_masks.shape[1] != gt_mask.shape[1]:
        pred_masks_resized = []
        for i in range(pred_masks.shape[-1]):
            mask = pred_masks[:, :, i]
            mask_resized = cv2.resize(mask.astype(np.uint8),
                                    (gt_mask.shape[1], gt_mask.shape[0]),
                                    interpolation=cv2.INTER_NEAREST)
            pred_masks_resized.append(mask_resized)
        pred_masks = np.stack(pred_masks_resized, axis=-1)

    
    if gt_class_ids.size == 0 or r['class_ids'].size == 0:
        continue

    
    AP, precisions, recalls, overlaps = compute_ap(
        gt_bbox,          # Ground truth bounding boxes
        gt_class_ids,     # Ground truth class IDs
        gt_mask,          # Ground truth masks
        r['rois'],        # Predicted bounding boxes
        r['class_ids'],   # Predicted class IDs
        r['scores'],      # Predicted scores
        pred_masks        # Predicted masks
    )

    APs.append(AP)

    
    gt_mask_binary = np.any(gt_mask > 0, axis=2).astype(np.int32)
    pred_mask_binary = np.any(pred_masks > 0, axis=2).astype(np.int32)

    
    gt_mask_flat = gt_mask_binary.flatten()
    pred_mask_flat = pred_mask_binary.flatten()

   
    all_true_labels.extend(gt_mask_flat)
    all_pred_labels.extend(pred_mask_flat)


all_true_labels = np.array(all_true_labels)
all_pred_labels = np.array(all_pred_labels)


if len(APs) > 0:
    
    mAP = np.mean(APs)
    print(f"Mean Average Precision (mAP): {mAP:.4f}")

    
    precision = precision_score(all_true_labels, all_pred_labels, average='binary')
    recall = recall_score(all_true_labels, all_pred_labels, average='binary')
    f1 = f1_score(all_true_labels, all_pred_labels, average='binary')

   
    print("\nOverall Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    
    print("\nPer-image AP Statistics:")
    print(f"Min AP: {np.min(APs):.4f}")
    print(f"Max AP: {np.max(APs):.4f}")
    print(f"Median AP: {np.median(APs):.4f}")
    print(f"Standard Deviation AP: {np.std(APs):.4f}")

    
    percentiles = [25, 50, 75, 90, 95]
    print("\nAP Percentiles:")
    for p in percentiles:
        print(f"{p}th percentile: {np.percentile(APs, p):.4f}")
else:
    print("No valid predictions found for evaluation")


import json
from datetime import datetime

metrics = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "mAP": float(mAP),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "num_images_evaluated": len(APs),
    "ap_statistics": {
        "min": float(np.min(APs)),
        "max": float(np.max(APs)),
        "median": float(np.median(APs)),
        "std": float(np.std(APs)),
        "percentiles": {
            str(p): float(np.percentile(APs, p)) for p in percentiles
        }
    }
}


with open('evaluation_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)
print("\nMetrics saved to 'evaluation_metrics.json'")

In [ ]:
import json
from datetime import datetime
import os

# Construire le chemin de sauvegarde
output_dir = "/content/images"
os.makedirs(output_dir, exist_ok=True)  # S'assure que le dossier existe
output_file = os.path.join(output_dir, "evaluation_metrics_model.json")

# Construire les métriques
metrics = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "mAP": float(mAP),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "num_images_evaluated": len(APs),
    "ap_statistics": {
        "min": float(np.min(APs)),
        "max": float(np.max(APs)),
        "median": float(np.median(APs)),
        "std": float(np.std(APs)),
        "percentiles": {
            str(p): float(np.percentile(APs, p)) for p in percentiles
        }
    }
}

# Sauvegarder les métriques dans un fichier JSON
with open(output_file, 'w') as f:
    json.dump(metrics, f, indent=4)

print(f"\nMetrics saved to '{output_file}'")


application fonction detection position du bébé

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io

def detect_baby_orientation(image_path, model):
    """Détecte l'orientation d'un bébé (face ou dos) dans une image.

    Args:
        image_path (str): Chemin de l'image à analyser.
        model (MaskRCNN): Modèle Mask R-CNN entraîné.

    Returns:
        str: "face" si le bébé est de face, "dos" si le bébé est de dos, "inconnu" sinon.
    """

    # Charger l'image
    image = skimage.io.imread(image_path)

    # Faire une prédiction avec le modèle
    results = inference_model.detect([image], verbose=0)
    r = results[0]

    # Vérifier les classes détectées
    class_ids = r['class_ids']

    # Définir les IDs de classe pour "face" et "nose"
    face_id = 1 # Assurez-vous que c'est l'ID de classe correct pour "face"
    nose_id = 2 # Assurez-vous que c'est l'ID de classe correct pour "nose"

    if face_id in class_ids or nose_id in class_ids:
        return "face"  # Bébé de face si "face" ou "nose" sont détectés
    elif face_id in class_ids:
        return "coté"   # Bébé de dos si seulement "face" est détecté
    else:
        return "dos"  # Orientation inconnue si ni "face" ni "nose" ne sont détectés


# Chemin du dossier contenant les images de test
test_images_dir = '/content/images/test'

# Parcourir les images de test et détecter l'orientation du bébé
for filename in os.listdir(test_images_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers image
        image_path = os.path.join(test_images_dir, filename)
        orientation = detect_baby_orientation(image_path, model)

        print(f"Image: {filename}, Orientation: {orientation}")

        # Afficher l'image avec l'orientation détectée (optionnel)
        image = skimage.io.imread(image_path)
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        # Ajouter une annotation pour l'orientation
        ax.text(10, 30, orientation, color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

        plt.show()

 # création du model et entrainement avec la deuxième configuration

In [ ]:
from mrcnn.model import MaskRCNN
config = BabyConfig_1()
# Créer le modèle en mode entraînement
model_1 = MaskRCNN(mode="training", config=config, model_dir='/content/logs')

In [ ]:
# Charger un modèle pré-entraîné (par exemple COCO)
model_1.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# Vérifier les types de données avant l'entraînement
for image_id in dataset_train.image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    assert image.dtype == np.uint8, f"Training image dtype is {image.dtype}"
    assert mask.dtype == np.uint8, f"Training mask dtype is {mask.dtype}"

In [ ]:
# Entraîner le modèle
model_1.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=75,
            layers='heads')


In [ ]:
history_path = os.path.join('/content', 'training_logs_model_1.txt')
with open(history_path, 'w') as log_file:
    log_file.write(str(model_1.keras_model.history.history))  


import matplotlib.pyplot as plt

loss = model_1.keras_model.history.history['loss']
val_loss = model_1.keras_model.history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

code inférence pour la deuxieme config des hyperparametres

In [ ]:
# Passer en mode inférence
inference_config = CustomConfig()
inference_config.GPU_COUNT = 1
inference_config.IMAGES_PER_GPU = 1

inference_config.BATCH_SIZE = 1 


model_1 = MaskRCNN(mode="inference", config=inference_config, model_dir='/content/logs')

import os
if os.path.exists('/content/logs/mask_rcnn_baby.h5'):
  model_1.load_weights('/content/logs/mask_rcnn_baby.h5', by_name=True)
else:
  print("Weight file not found. Please ensure the model has been trained and the weights are saved to the correct location.")
  

# Charger une image pour effectuer des prédictions
image_id = dataset_val.image_ids[0]
image = dataset_val.load_image(image_id)
mask, class_ids = dataset_val.load_mask(image_id)

# Effectuer la prédiction
results = model_1.detect([image], verbose=1)
r = results[0]

# Afficher les résultats
import mrcnn.visualize
mrcnn.visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                  dataset_val.class_names, r['scores'])

In [ ]:

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

class InferenceConfig(Config):
    NAME = "Baby"
    NUM_CLASSES = 1 + 2  
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_model_1 = MaskRCNN(mode="inference", config=inference_config, model_dir='/content')


inference_model_1.load_weights(model_1.find_last(), by_name=True)

true_labels = []
pred_labels = []

for image_id in dataset_test.image_ids:
    
    true_masks, true_classes = dataset_test.load_mask(image_id)

    
    image = dataset_test.load_image(image_id)
    pred = inference_model_1.detect([image], verbose=0)[0]

    
    if len(true_classes) == len(pred['class_ids']):
        true_labels.extend(true_classes)
        pred_labels.extend(pred['class_ids'])
    else:
        
        true_labels.extend(true_classes)
        pred_labels.extend(pred['class_ids'])

       
        if len(true_classes) > len(pred['class_ids']):
            pred_labels.extend([0] * (len(true_classes) - len(pred['class_ids'])))
        elif len(pred['class_ids']) > len(true_classes):
            true_labels.extend([0] * (len(pred['class_ids']) - len(true_classes)))


cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1]) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['background', 'baby'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


from mrcnn.visualize import display_instances
from mrcnn import visualize

import random


for i in random.sample(list(dataset_test.image_ids), 5):
    image = dataset_test.load_image(i)  
    results = inference_model_1.detect([image], verbose=0)  

    
    visualize.display_instances(
        image,
        results[0]['rois'],
        results[0]['masks'],
        results[0]['class_ids'],
        dataset_test.class_names,
        results[0]['scores']
    )


In [ ]:
from mrcnn.utils import compute_ap
import numpy as np


APs = []


for image_id in dataset_test.image_ids:
    
    image = dataset_test.load_image(image_id)

    
    gt_mask, gt_class_ids = dataset_test.load_mask(image_id)

    
    gt_bbox = utils.extract_bboxes(gt_mask)

    
    results = inference_model_1.detect([image], verbose=0)
    r = results[0]

    
    if r['masks'].size == 0:
        continue

    
    pred_masks = r['masks']

    
    if pred_masks.shape[0] != gt_mask.shape[0] or pred_masks.shape[1] != gt_mask.shape[1]:
        pred_masks_resized = []
        for i in range(pred_masks.shape[-1]):
            mask = pred_masks[:, :, i]
            mask_resized = cv2.resize(mask.astype(np.uint8),
                                    (gt_mask.shape[1], gt_mask.shape[0]),
                                    interpolation=cv2.INTER_NEAREST)
            pred_masks_resized.append(mask_resized)
        pred_masks = np.stack(pred_masks_resized, axis=-1)

    
    if gt_class_ids.size == 0 or r['class_ids'].size == 0:
        continue

    
    AP, precisions, recalls, overlaps = compute_ap(
        gt_bbox,          # Ground truth bounding boxes
        gt_class_ids,     # Ground truth class IDs
        gt_mask,          # Ground truth masks
        r['rois'],        # Predicted bounding boxes
        r['class_ids'],   # Predicted class IDs
        r['scores'],      # Predicted scores (was missing in original)
        pred_masks        # Predicted masks
    )

    APs.append(AP)


if len(APs) > 0:
    mAP = np.mean(APs)
    print(f"Mean Average Precision (mAP): {mAP:.4f}")
else:
    print("No valid predictions found for evaluation")


In [ ]:
from mrcnn.utils import compute_ap
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import cv2


APs = []
all_true_labels = []
all_pred_labels = []


for image_id in dataset_test.image_ids:
    
    image = dataset_test.load_image(image_id)

    
    gt_mask, gt_class_ids = dataset_test.load_mask(image_id)

    
    gt_bbox = utils.extract_bboxes(gt_mask)

    
    results = inference_model_1.detect([image], verbose=0)
    r = results[0]

    
    if r['masks'].size == 0:
        continue

    
    pred_masks = r['masks']

    
    if pred_masks.shape[0] != gt_mask.shape[0] or pred_masks.shape[1] != gt_mask.shape[1]:
        pred_masks_resized = []
        for i in range(pred_masks.shape[-1]):
            mask = pred_masks[:, :, i]
            mask_resized = cv2.resize(mask.astype(np.uint8),
                                    (gt_mask.shape[1], gt_mask.shape[0]),
                                    interpolation=cv2.INTER_NEAREST)
            pred_masks_resized.append(mask_resized)
        pred_masks = np.stack(pred_masks_resized, axis=-1)

    
    if gt_class_ids.size == 0 or r['class_ids'].size == 0:
        continue

    
    AP, precisions, recalls, overlaps = compute_ap(
        gt_bbox,          # Ground truth bounding boxes
        gt_class_ids,     # Ground truth class IDs
        gt_mask,          # Ground truth masks
        r['rois'],        # Predicted bounding boxes
        r['class_ids'],   # Predicted class IDs
        r['scores'],      # Predicted scores
        pred_masks        # Predicted masks
    )

    APs.append(AP)

    
    gt_mask_binary = np.any(gt_mask > 0, axis=2).astype(np.int32)
    pred_mask_binary = np.any(pred_masks > 0, axis=2).astype(np.int32)

    
    gt_mask_flat = gt_mask_binary.flatten()
    pred_mask_flat = pred_mask_binary.flatten()

    
    all_true_labels.extend(gt_mask_flat)
    all_pred_labels.extend(pred_mask_flat)


all_true_labels = np.array(all_true_labels)
all_pred_labels = np.array(all_pred_labels)


if len(APs) > 0:
    
    mAP = np.mean(APs)
    print(f"Mean Average Precision (mAP): {mAP:.4f}")

    
    precision = precision_score(all_true_labels, all_pred_labels, average='binary')
    recall = recall_score(all_true_labels, all_pred_labels, average='binary')
    f1 = f1_score(all_true_labels, all_pred_labels, average='binary')

    
    print("\nOverall Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    
    print("\nPer-image AP Statistics:")
    print(f"Min AP: {np.min(APs):.4f}")
    print(f"Max AP: {np.max(APs):.4f}")
    print(f"Median AP: {np.median(APs):.4f}")
    print(f"Standard Deviation AP: {np.std(APs):.4f}")

    
    percentiles = [25, 50, 75, 90, 95]
    print("\nAP Percentiles:")
    for p in percentiles:
        print(f"{p}th percentile: {np.percentile(APs, p):.4f}")
else:
    print("No valid predictions found for evaluation")


import json
from datetime import datetime

metrics = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "mAP": float(mAP),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "num_images_evaluated": len(APs),
    "ap_statistics": {
        "min": float(np.min(APs)),
        "max": float(np.max(APs)),
        "median": float(np.median(APs)),
        "std": float(np.std(APs)),
        "percentiles": {
            str(p): float(np.percentile(APs, p)) for p in percentiles
        }
    }
}

# Save metrics to JSON file
with open('evaluation_metrics_1.json', 'w') as f:
    json.dump(metrics, f, indent=4)
print("\nMetrics saved to 'evaluation_metrics_1.json'")

In [ ]:
import json
from datetime import datetime
import os

# Construire le chemin de sauvegarde
output_dir = "/content/images"
os.makedirs(output_dir, exist_ok=True)  # S'assure que le dossier existe
output_file = os.path.join(output_dir, "evaluation_metrics_model_1.json")

# Construire les métriques
metrics = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "mAP": float(mAP),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "num_images_evaluated": len(APs),
    "ap_statistics": {
        "min": float(np.min(APs)),
        "max": float(np.max(APs)),
        "median": float(np.median(APs)),
        "std": float(np.std(APs)),
        "percentiles": {
            str(p): float(np.percentile(APs, p)) for p in percentiles
        }
    }
}

# Sauvegarder les métriques dans un fichier JSON
with open(output_file, 'w') as f:
    json.dump(metrics, f, indent=4)

print(f"\nMetrics saved to '{output_file}'")


fonction qui permet la detction de la position du bébé

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io

def detect_baby_orientation(image_path, model):
    """Détecte l'orientation d'un bébé (face ou dos) dans une image.

    Args:
        image_path (str): Chemin de l'image à analyser.
        model (MaskRCNN): Modèle Mask R-CNN entraîné.

    Returns:
        str: "face" si le bébé est de face, "dos" si le bébé est de dos, "inconnu" sinon.
    """

    # Charger l'image
    image = skimage.io.imread(image_path)

    # Faire une prédiction avec le modèle
    results = inference_model_1.detect([image], verbose=0)
    r = results[0]

    # Vérifier les classes détectées
    class_ids = r['class_ids']

    # Définir les IDs de classe pour "face" et "nose"
    face_id = 1 # Assurez-vous que c'est l'ID de classe correct pour "face"
    nose_id = 2 # Assurez-vous que c'est l'ID de classe correct pour "nose"

    if face_id in class_ids or nose_id in class_ids:
        return "face"  # Bébé de face si "face" et "nose" sont détectés
    elif face_id in class_ids:
        return "coté"   # Bébé de dos si seulement "face" est détecté
    else:
        return "dos"  # Orientation inconnue si ni "face" ni "nose" ne sont détectés


# Chemin du dossier contenant les images de test
test_images_dir = '/content/images/test'

# Parcourir les images de test et détecter l'orientation du bébé
for filename in os.listdir(test_images_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers image
        image_path = os.path.join(test_images_dir, filename)
        orientation = detect_baby_orientation(image_path, model_1)

        print(f"Image: {filename}, Orientation: {orientation}")

        # Afficher l'image avec l'orientation détectée (optionnel)
        image = skimage.io.imread(image_path)
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        # Ajouter une annotation pour l'orientation
        ax.text(10, 30, orientation, color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

        plt.show()

avec application sur une video

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def detect_baby_orientation(image, model):
    """
    Détecte l'orientation du bébé dans une image.
    Retourne l'orientation détectée et les résultats du modèle (bounding boxes, etc.).
    """
    # Convertir l'image de BGR à RGB pour le modèle Mask R-CNN
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Faire une prédiction avec le modèle
    results = model.detect([rgb_image], verbose=0)
    r = results[0]

    # Ici vous pouvez définir la logique d'orientation de votre bébé en fonction des classes détectées.
    # (Déterminez l'orientation selon les résultats du modèle)
    class_ids = r['class_ids']

    face_id = 1  # Exemple d'ID de classe pour "face"
    nose_id = 2  # Exemple d'ID de classe pour "nose"

    if face_id in class_ids or nose_id in class_ids:
        orientation = "face"  # Bébé de face
    elif face_id in class_ids:
        orientation = "dos"  # Bébé de dos
    else:
        orientation = "inconnu"  # Orientation inconnue

    return orientation, r

def process_video(video_path, model, output_path=None):
    """
    Traite une vidéo pour détecter la position du bébé image par image.
    Enregistre la vidéo de sortie avec les annotations (bounding boxes et orientation).
    """
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    # Vérifier si la vidéo s'ouvre correctement
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la vidéo.")
        return

    # Obtenir les dimensions et le FPS de la vidéo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Définir le writer si un fichier de sortie est spécifié
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour le fichier de sortie
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir l'image de BGR à RGB pour l'input du modèle
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Détecter l'orientation du bébé
        orientation, results = detect_baby_orientation(rgb_frame, model)

        # Dessiner les bounding boxes et afficher l'orientation
        for i, bbox in enumerate(results['rois']):
            y1, x1, y2, x2 = bbox
            label = orientation
            color = (0, 255, 0) if label == "face" else (0, 0, 255)

            # Dessiner la bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Ajouter le texte pour l'orientation
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Afficher la frame dans Colab
        cv2_imshow(frame)

        # Enregistrer dans le fichier de sortie si nécessaire
        if output_path:
            out.write(frame)

        # Arrêter avec la touche 'q' (nécessaire uniquement pour exécutions locales)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Chemin de la vidéo d'entrée et du fichier de sortie
video_input_path = '/content/video.mp4'
video_output_path = '/content/output_video.mp4'

# Traiter la vidéo avec le modèle
process_video(video_input_path, inference_model_1, output_path=video_output_path)


Fonction qui permet de determiner la position du bébé sur une vidéo

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
# Import cv2_imshow from google.colab.patches
from google.colab.patches import cv2_imshow

# ... (other functions remain the same) ...

def process_video(video_path, model, output_path=None):
    """
    Traite une vidéo pour détecter la position du bébé image par image.

    Args:
        video_path (str): Chemin de la vidéo à analyser.
        model (MaskRCNN): Modèle Mask R-CNN entraîné.
        output_path (str): Chemin du fichier vidéo de sortie (facultatif).

    Returns:
        None
    """
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    # Vérifier si la vidéo s'ouvre correctement
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la vidéo.")
        return

    # Obtenir les dimensions de la vidéo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Définir le writer si un fichier de sortie est spécifié
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour le fichier de sortie
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir l'image de BGR (par défaut OpenCV) à RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Détecter l'orientation du bébé
        orientation, results = detect_baby_orientation(rgb_frame, model)

        # Dessiner les bounding boxes et afficher l'orientation
        for i, bbox in enumerate(results['rois']):
            y1, x1, y2, x2 = bbox
            label = orientation
            color = (0, 255, 0) if label == "face" else (0, 0, 255)

            # Dessiner la bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Ajouter le texte pour l'orientation
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Afficher la frame using cv2_imshow()
        cv2_imshow(frame)

        # Enregistrer dans le fichier de sortie si nécessaire
        if output_path:
            out.write(frame)

        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()


# Chemin de la vidéo d'entrée et du fichier de sortie
video_input_path = '/content/video.mp4'
video_output_path = '/content/output_video.mp4'

# Traiter la vidéo avec le modèle
process_video(video_input_path, inference_model_1, output_path=video_output_path)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def process_video(video_path, model, output_path=None):
    """
    Traite une vidéo pour détecter la position du bébé image par image.

    Args:
        video_path (str): Chemin de la vidéo à analyser.
        model (MaskRCNN): Modèle Mask R-CNN entraîné.
        output_path (str): Chemin du fichier vidéo de sortie (facultatif).

    Returns:
        None
    """
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    # Vérifier si la vidéo s'ouvre correctement
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la vidéo.")
        return

    # Obtenir les dimensions et le FPS de la vidéo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Définir le writer si un fichier de sortie est spécifié
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour le fichier de sortie
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir l'image de BGR (par défaut OpenCV) à RGB pour le modèle
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Détecter l'orientation du bébé
        orientation, results = detect_baby_orientation(rgb_frame, model)

        # Dessiner les bounding boxes et afficher l'orientation
        for i, bbox in enumerate(results['rois']):
            y1, x1, y2, x2 = bbox
            label = orientation
            color = (0, 255, 0) if label == "face" else (0, 0, 255)

            # Dessiner la bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Ajouter le texte pour l'orientation
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Afficher la frame dans Colab
        cv2_imshow(frame)

        # Enregistrer dans le fichier de sortie si nécessaire
        if output_path:
            out.write(frame)

        # Arrêter avec la touche 'q' (nécessaire uniquement pour exécutions locales)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()


# Chemin de la vidéo d'entrée et du fichier de sortie
video_input_path = '/content/video.mp4'
video_output_path = '/content/output_video.mp4'

# Traiter la vidéo avec le modèle
process_video(video_input_path, inference_model_1, output_path=video_output_path)


# Comparaison des deux models

## Première comparaison avec les logs d'entrainement

In [ ]:
import re

def parse_logs(file_path):
    """
    Extrait les métriques à partir d'un fichier de logs.
    Supposons que chaque ligne de log contient une mention claire de la métrique d'intérêt (par exemple, 'loss', 'accuracy').
    """
    metrics = []
    with open(file_path, 'r') as f:
        for line in f:
            # Utiliser une regex pour détecter des métriques spécifiques
            match = re.search(r"loss: ([0-9]*\.[0-9]+)|accuracy: ([0-9]*\.[0-9]+)", line)
            if match:
                loss = float(match.group(1)) if match.group(1) else None
                accuracy = float(match.group(2)) if match.group(2) else None
                metrics.append({"loss": loss, "accuracy": accuracy})
    return metrics

def compare_models(log1, log2, metric="accuracy"):
    """
    Compare deux modèles sur une métrique spécifique.
    :param log1: Liste de dictionnaires (métriques du modèle 1).
    :param log2: Liste de dictionnaires (métriques du modèle 2).
    :param metric: La métrique à comparer ('accuracy' ou 'loss').
    :return: Le modèle gagnant et les moyennes des métriques.
    """
    model1_values = [entry[metric] for entry in log1 if entry[metric] is not None]
    model2_values = [entry[metric] for entry in log2 if entry[metric] is not None]

    model1_avg = sum(model1_values) / len(model1_values) if model1_values else float('inf')
    model2_avg = sum(model2_values) / len(model2_values) if model2_values else float('inf')

    if metric == "loss":
        winner = "Model 1" if model1_avg < model2_avg else "Model 2"
    else:  # Pour 'accuracy' et autres métriques où un score plus élevé est meilleur
        winner = "Model 1" if model1_avg > model2_avg else "Model 2"

    return winner, model1_avg, model2_avg

# Chemins des fichiers de logs
file1 = "/content/training_logs_model.txt"
file2 = "/content/training_logs_model_1.txt"

# Analyse des fichiers
logs_model1 = parse_logs(file1)
logs_model2 = parse_logs(file2)

# Comparaison
metric_to_compare = "accuracy"  # ou "loss"
winner, avg_model1, avg_model2 = compare_models(logs_model1, logs_model2, metric=metric_to_compare)

print(f"Métrique comparée : {metric_to_compare}")
print(f"Moyenne modèle 1 : {avg_model1}")
print(f"Moyenne modèle 2 : {avg_model2}")
print(f"Modèle le plus performant : {winner}")


## Deuxième comparaison cette fois avec les métriques d'évaluation

In [ ]:
import json

def load_metrics(file_path):
    """
    Charge les métriques à partir d'un fichier JSON.
    :param file_path: Chemin vers le fichier JSON.
    :return: Dictionnaire contenant les métriques.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

def compare_metrics(metrics1, metrics2, keys_to_compare=None):
    """
    Compare les métriques de deux modèles.
    :param metrics1: Dictionnaire des métriques du modèle 1.
    :param metrics2: Dictionnaire des métriques du modèle 2.
    :param keys_to_compare: Liste des clés à comparer. Compare tout si None.
    :return: Résultat de la comparaison.
    """
    if keys_to_compare is None:
        keys_to_compare = ["mAP", "precision", "recall", "f1_score"]

    results = {}
    for key in keys_to_compare:
        value1 = metrics1.get(key, None)
        value2 = metrics2.get(key, None)

        if value1 is None or value2 is None:
            results[key] = "Indisponible pour comparaison"
        else:
            # Déterminer le meilleur modèle pour chaque métrique
            if value1 > value2:
                results[key] = "Model 1 est meilleur"
            elif value1 < value2:
                results[key] = "Model 2 est meilleur"
            else:
                results[key] = "Égalité"

    return results

# Chemins des fichiers JSON
file1 = "/content/evaluation_metrics.json"
file2 = "/content/evaluation_metrics_1.json"

# Chargement des métriques
metrics_model1 = load_metrics(file1)
metrics_model2 = load_metrics(file2)

# Comparaison
comparison_results = compare_metrics(metrics_model1, metrics_model2)

# Afficher les résultats
print("\nRésultats de la comparaison :")
for metric, result in comparison_results.items():
    print(f"{metric}: {result}")

# Identifier le modèle globalement meilleur (par exemple, basé sur le F1-score ou mAP)
key_priority = "f1_score"  # Priorité sur F1-score
if metrics_model1[key_priority] > metrics_model2[key_priority]:
    print("\nGlobalement, Model 1 est meilleur basé sur le F1-score.")
elif metrics_model1[key_priority] < metrics_model2[key_priority]:
    print("\nGlobalement, Model 2 est meilleur basé sur le F1-score.")
else:
    print("\nGlobalement, les deux modèles sont équivalents sur le F1-score.")
